In [91]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [92]:
farms = pd.read_csv("../data-mining/farms.csv")
rubrics = pd.read_csv("../data-mining/rubrics.csv")
stocks = pd.read_csv("../data-mining/stocks.csv")

## Add departement column

In [93]:
def departement_of(code_postal: int) -> int:
    departement = int(str(code_postal)[:2])

    # dom_tom departement code is 3 digits-long instead of 2
    dom_tom: bool = departement == 97
    if dom_tom:
        departement = int(str(code_postal)[:3])

    return departement

farms["departement"] = farms.codePostal.apply(departement_of)

In [94]:
farms_stocks = farms.merge(stocks, left_on="name", right_on="raison_sociale")

In [95]:
farms_stocks = farms_stocks[["name", "rubric_id", "quantite", "unite", "departement", "type"]]
farms_stocks = farms_stocks[(farms_stocks.unite == "u") | (farms_stocks.unite == "Animaux-Eq")]
farms_stocks

,name,rubric_id,quantite,unite,departement,type
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq,89,PORCS
5,GAEC FERME H ET L,2101,210.0,u,50,BOVINS
6,BINOUDIERE (GAEC DE LA),2102,498.0,Animaux-Eq,50,BOVINS
8,BINOUDIERE (GAEC DE LA),2101,60.0,u,50,BOVINS
9,BINOUDIERE (GAEC DE LA),2101,80.0,u,50,BOVINS
...,...,...,...,...,...,...
23680,DE LA CLINERIE (GAEC) LEFEVRE Christian,2102,1059.0,Animaux-Eq,45,VOLAILLES
23682,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,Animaux-Eq,45,VOLAILLES
23684,DES JOURNETS (SCEA),2102,1857.0,Animaux-Eq,41,PORCS
23685,EARL DES MUSSETS,2111,40000.0,u,41,VOLAILLES


In [96]:
farms_stocks = pd.DataFrame({'quantite_mean' : farms_stocks.groupby(
    ["name", "rubric_id", "unite", "departement", "type"])['quantite'].mean()}).reset_index()

In [97]:
farms_stocks

,name,rubric_id,unite,departement,type,quantite_mean
0,2 L'oiselière (GAEC),2101,u,50,BOVINS,130.000000
1,33 RUE DU BOCAGE (GAEC),2102,Animaux-Eq,50,PORCS,747.000000
2,4 CANTONS DUBRON (GAEC DES ),2101,u,62,BOVINS,215.000000
3,4 CANTONS DUBRON (GAEC DES ),2102,Animaux-Eq,62,BOVINS,384.000000
4,4 CANTONS DUBRON (GAEC DES ),2111,Animaux-Eq,62,BOVINS,22500.000000
...,...,...,...,...,...,...
10881,earl la roche marteau,2101,u,36,BOVINS,180.000000
10882,gene +,2102,Animaux-Eq,62,PORCS,1478.200000
10883,quatres sites (GAEC des),2101,u,19,BOVINS,107.666667
10884,quatres sites (GAEC des),2102,Animaux-Eq,19,BOVINS,830.000000


In [98]:
farms_stocks["departement_unite_rank"] = farms_stocks.groupby(
    ["unite", "departement"])['quantite_mean'].rank(ascending=False)

In [99]:
farms_stocks["france_unite_rank"] = farms_stocks.groupby(
    ["unite"]
)["quantite_mean"].rank(ascending=False)

In [100]:
farms_stocks.sort_values(by=["france_unite_rank"], inplace=True)

In [105]:
farms_stocks

,name,rubric_id,unite,departement,type,quantite_mean,departement_unite_rank,france_unite_rank
7502,HUBBARD,2112,u,45,VOLAILLES,2073200.0,1.0,1.0
8192,MARANGER NADINE,2111,Animaux-Eq,36,VOLAILLES,216000.0,1.0,1.0
5423,GAEC DE TREGUEE,2111,Animaux-Eq,29,VOLAILLES,92138.0,1.0,2.0
10592,SOCAVIC,2112,u,40,VOLAILLES,1845000.0,1.0,2.0
1082,EARL AVI-ROULAIS,2111,Animaux-Eq,44,VOLAILLES,81600.0,1.0,3.0
...,...,...,...,...,...,...,...,...
12,ABATTOIRS DU GEVAUDAN,2102,Animaux-Eq,48,BOVINS,64.0,5.0,6513.0
9436,SCEA APPORCS,2102,Animaux-Eq,79,PORCS,60.0,110.0,6514.0
10649,Sté d'Exploitation Abattoir de Brioude,2102,Animaux-Eq,43,BOVINS,50.0,32.0,6515.0
2402,EARL DU GRAND BOIS,2102,Animaux-Eq,59,PORCS,0.0,200.0,6516.5


## Export to json

In [106]:
farms_stocks.to_json("4.json", orient="records")